In [1]:
!uv pip install -U langgraph langchain-anthropic tavily-python python-dotenv

from dotenv import load_dotenv
load_dotenv()

import os

# LangSmith config (project already created as infosys-springboard-m1)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "infosys-springboard-m1")


Resolved 41 packages in 500ms
Audited 41 packages in 58ms


In [3]:
!pip install groq


  Using cached groq-1.0.0-py3-none-any.whl.metadata (16 kB)
Using cached groq-1.0.0-py3-none-any.whl (138 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import groq
groq.__version__


'1.0.0'

In [5]:
from agent_state import AgentState, Todo
from tools import writetodos_tool as write_todos
from langgraph.graph import StateGraph, END
from typing import cast

def brain_node(state: AgentState) -> AgentState:
    messages = state.get("messages", [])
    todos = state.get("todos", [])

    if not todos and messages:
        user_request = messages[-1]
        new_todos = write_todos(user_request)
        state["todos"] = new_todos
        messages.append("Planner: TODO list created.")
    else:
        messages.append("Planner: TODOs already exist.")

    state["messages"] = messages
    return state

builder = StateGraph(AgentState)
builder.add_node("brain", brain_node)
builder.set_entry_point("brain")
builder.add_edge("brain", END)
graph = builder.compile()


In [6]:
from agent_state import AgentState
from typing import cast

initial_state: AgentState = {
    "messages": ["Conduct a deep dive into renewable energy trends."],
    "todos": [],
}

final_state = cast(AgentState, graph.invoke(initial_state))
final_state


{'messages': ['Conduct a deep dive into renewable energy trends.',
  'Planner: TODO list created.'],
 'todos': [{'id': 1,
   'description': 'Research current global renewable energy production capacity',
   'status': 'pending'},
  {'id': 2,
   'description': 'Analyze recent trends in solar panel efficiency and cost',
   'status': 'pending'},
  {'id': 3,
   'description': 'Identify top 5 countries by renewable energy investment in 2022',
   'status': 'pending'},
  {'id': 4,
   'description': 'Gather data on the current state of offshore wind energy development',
   'status': 'pending'},
  {'id': 5,
   'description': 'Compare the cost-effectiveness of different types of renewable energy sources',
   'status': 'pending'},
  {'id': 6,
   'description': 'Research the impact of government policies on renewable energy adoption',
   'status': 'pending'},
  {'id': 7,
   'description': 'Compile a list of key players in the renewable energy industry',
   'status': 'pending'}]}

In [4]:
import os
import json
from typing import List
from dotenv import load_dotenv
from anthropic import Anthropic
from agent_state import Todo

load_dotenv()
client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

def writetodos_tool(user_request: str) -> List[Todo]:
    """
    Use Claude to break a complex goal into TODOs.
    """
    prompt = f"""
You are a planning assistant for an autonomous research agent.

Goal:
\"\"\"{user_request}\"\"\"

Break this goal into 4–7 clear, concrete sub-tasks that a smart intern could execute.
Return ONLY valid JSON in this format:

{{
  "todos": [
    {{"id": 1, "description": "first task", "status": "pending"}},
    {{"id": 2, "description": "second task", "status": "pending"}}
  ]
}}
"""

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=400,
        messages=[{"role": "user", "content": prompt}],
    )

    text = message.content[0].text.strip()

    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        start = text.find("{")
        end = text.rfind("}")
        data = json.loads(text[start:end + 1])

    todos: List[Todo] = data["todos"]
    return todos


In [8]:
high_horizon_prompts = [
    "Conduct a deep dive into renewable energy trends in India and draft an executive summary for policymakers.",
    "Research the impact of 2026 carbon taxes on European manufacturing and draft a risk assessment.",
    "Design a roadmap for adopting AI copilots in a mid-sized IT services company over the next 3 years.",
    "Analyze trends in EV battery recycling technologies and identify top 5 business opportunities.",
    "Study global data privacy regulations and propose a compliance checklist for a SaaS startup.",
    "Evaluate the risks and benefits of using generative AI for customer support in banking.",
    "Research green logistics strategies for e-commerce companies in Southeast Asia.",
    "Investigate cybersecurity threats for remote work infrastructures and draft a mitigation plan.",
    "Explore opportunities for AI-powered education tools in Indian higher education.",
    "Assess the business impact of quantum computing advances over the next decade.",
]

len(high_horizon_prompts), high_horizon_prompts[0]


(10,
 'Conduct a deep dive into renewable energy trends in India and draft an executive summary for policymakers.')

In [9]:
from typing import cast
from agent_state import AgentState

results = []

for i, prompt in enumerate(high_horizon_prompts, start=1):
    state: AgentState = {"messages": [prompt], "todos": []}
    final_state = cast(AgentState, graph.invoke(state))
    results.append((prompt, final_state["todos"]))

len(results)


10

In [10]:
for prompt, todos in results:
    print("PROMPT:", prompt)
    print("TODOs:")
    for todo in todos:
        print(f"  [{todo['id']}] {todo['description']} (status={todo['status']})")
    print("-" * 80)


PROMPT: Conduct a deep dive into renewable energy trends in India and draft an executive summary for policymakers.
TODOs:
  [1] Research current renewable energy policies in India (status=pending)
  [2] Gather data on India's renewable energy production and consumption trends (status=pending)
  [3] Identify key players in India's renewable energy sector (government agencies, companies, etc.) (status=pending)
  [4] Analyze the impact of government incentives and subsidies on renewable energy adoption in India (status=pending)
  [5] Review existing literature on renewable energy trends in India (academic papers, reports, etc.) (status=pending)
  [6] Compile a list of potential interviewees for the project (experts, industry leaders, etc.) (status=pending)
  [7] Draft an outline for the executive summary (status=pending)
--------------------------------------------------------------------------------
PROMPT: Research the impact of 2026 carbon taxes on European manufacturing and draft a ri

In [11]:
import json

with open("milestone1_todos.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)
